## This is a temporary testing notebook for working on the automation of prove_by_cases

The content will eventually be deleted (with some bits of the content transferred to the enumeration.\_demonstrations\_.ipynb notebook).

In [ ]:
import proveit
from proveit import (var_range, ExprTuple, ExprRange, InstantiationFailure, Operation,
                     ProofFailure, used_vars, free_vars, Function)
from proveit import a, b, c, d, e, i, l, m, n, x, y, P, Px, Q
from proveit import A, B, C, D, E
from proveit.core_expr_types import (a_1_to_n)
from proveit.logic import And, Boolean, Implies, TRUE, FALSE
from proveit.logic import (Equals, Forall, InSet, Not, NotEquals, NotInSet,
                           Or, ProperSubset, Set, SubsetEq)
from proveit.logic.sets.enumeration import (
        true_for_each_is_true_for_all, true_for_each_then_true_for_all,
        true_for_each_then_true_for_all_conditioned)
from proveit.numbers import zero, one, two, three, four, five, six, seven, num, greater, Less

### Two theorems to be instantiated:

In [ ]:
true_for_each_then_true_for_all

In [ ]:
true_for_each_then_true_for_all_conditioned

#### The 1st one is simpler, so we play with that one first. The BooleanSet.fold_as_forall() method we are adapting is called on a Boolean set along with a forall argument to be proven based on the forall instance expr being true, or assumed to be true, for each Boolean in the underlying set. Our prove_by_cases generalizes this from a Boolean set to any enumerated Set.

So let's define a simple set:

In [ ]:
# A 3-element set of (literal) integers:
set_123 = Set(one, two, three)

In [ ]:
set_13 = Set(one, three)

#### A simple forall statement we might conceivable want to conclude would be something like this, with a very simple predicate that is clearly true for all elements of the set:

In [ ]:
forall_example_01 = Forall(y, Less(y, four), condition=InSet(y, set_123))

#### Out effort to automate the process is successful:

In [ ]:
forall_example_01.prove()

#### A more complicated forall statement involving a condition $Q(x)$ that clearly implies the instance expression $P(x)$:

In [ ]:
forall_example_02 = Forall(y, Less(y, four), domain = set_123, conditions=[Less(y, zero)])

In [ ]:
forall_example_02.prove()

In [ ]:
Less(y, one).prove(assumptions=[Less(y, zero)])

In [ ]:
forall_example_03 = Forall(y, Less(y, one), domain = set_123, conditions=[Less(y, zero)])

In [ ]:
# this is an interesting case: the condition y < 0 is never true
# and thus the actual set being considered is an empty set … sort of
# in any case, y < 0 => y < 1 regardless of the values used for y, yes?
# And since 1 < 0 is False, the condition 1 < 0 => 1 < 1 is True, yes?
forall_example_03.prove(assumptions=[Implies(Less(one, zero), Less(one, one))])

In [ ]:
forall_example_04 = Forall(y, Less(y, five), domain = set_123, conditions=[greater(y, zero)])

In [ ]:
forall_example_04.prove()

In [ ]:
forall_example_05 = Forall(y, greater(y, two), domain = set_13, conditions=[greater(y, one)])

In [ ]:
# greater(one, one).disprove()

In [ ]:
forall_example_05.prove(assumptions=[Not(greater(one, one))])

### Testing to work out instantiation details

#### Analogous to the BooleanSet.fold_as_forall(self, forall_stmt, assumptions=USE_DEFAULTS) method, we'll want to access and use the following pieces of the user-supplied forall_stmt:

In [ ]:
# forall conditions
forall_example_01.conditions

In [ ]:
forall_example_01.condition.domain.operands.length()

In [ ]:
forall_example_01.conditions[0].domain.operands.length()

In [ ]:
# length of the forall conditions -- in this case, the length
# is just 1, which puts us in the simple ELSE case
len(forall_example_01.conditions)

In [ ]:
# for instance_var
forall_example_01.instance_var

In [ ]:
# for instance_expr
forall_example_01.instance_expr

In [ ]:
forall_example_01.first_domain()

#### A reminder of the theorem we're instantiating:

In [ ]:
true_for_each_then_true_for_all

n = size of set = 3 (can be found from len(set.elements)<br/>
a1,...,an = our set = {1, 2, 3} = set_123<br/>
Pred(x) = predicate = (x < 4) (notice the fxn name might not match theorem's P, and arg might not match thm's 'a'

In [ ]:
true_for_each_then_true_for_all.instance_var

#### We need to construct some things to make the instantiation substitutions work. For example, 

In [ ]:
n_sub = forall_example_01.domain.operands.length()

In [ ]:
# the following is interesting … is the length() operation above already giving a num() version?
# var_range_update = var_range(a, one, num(n_sub))
var_range_update = var_range(a, one, n_sub)

In [ ]:
# var_range_sub = set_123.elements
var_range_sub = forall_example_01.condition.domain.elements

In [ ]:
# BooleanSet.fold_as_forall() constructs P(x) -- not clear why we don't just import?
# Because we need to construct P(unknown), where the unknown is the instance var supplied
# by the user in the forall argument. The function variable P will match, but we need
# the arg variable to match up
Px = Function(P, forall_example_01.instance_var)

In [ ]:
# For P(x) we substitute the user-supplied instance expression
Px_sub = forall_example_01.instance_expr

In [ ]:
x_sub = forall_example_01.instance_var

In [ ]:
# must include explicit num_forall_eliminations arg!
inst_example_01 = true_for_each_then_true_for_all.instantiate(
    {n:three, ExprTuple(var_range_update):var_range_sub, x:x_sub, Px:Px_sub },
    num_forall_eliminations=3)

### Testing to work out instantiation details for 2nd Theorem

A reminder of the theorem we're instantiating:

In [ ]:
true_for_each_then_true_for_all

In [ ]:
true_for_each_then_true_for_all_conditioned

We need a slightly different forall example:

In [ ]:
forall_example_02 = Forall(y, Less(y, one), domain = set_123, conditions=[Less(y, zero)])

In [ ]:
# testing how one might specify the domain
forall_example_03 = Forall(y, Less(y, one), conditions=[InSet(y, set_123), Less(y, zero)])

In [ ]:
forall_example_02.conditions

In [ ]:
forall_example_02.domain

In [ ]:
forall_example_03.domain

Now though, we'll use $Q(x) = [x < 0]$ and $P(x) = [x < 1]$. Notice then that $Q(x) \Rightarrow P(x)$.

In [ ]:
n_sub = forall_example_02.domain.operands.length()

In [ ]:
var_range_update = var_range(a, one, n_sub)

In [ ]:
# var_range_sub = set_123.elements
var_range_sub = forall_example_01.condition.domain.elements

In [ ]:
# BooleanSet.fold_as_forall() constructs P(x) -- why don't jwe ust import?
# Because we need to construct P(unknown), where the unknown is the instance var supplied
# by the user in the forall argument. The function variable P will match, but we need
# the arg variable to match up
Px = Function(P, forall_example_02.instance_var)

In [ ]:
# For P(x) we substitute the user-supplied instance expression
Px_sub = forall_example_02.instance_expr

In [ ]:
Qx = Function(Q, forall_example_02.instance_var)

In [ ]:
Qx_sub = forall_example_02.conditions[1]

In [ ]:
x_sub = forall_example_02.instance_var

In [ ]:
true_for_each_then_true_for_all_conditioned.instantiate(
    {n: n_sub, ExprTuple(var_range_update):var_range_sub}, num_forall_eliminations=2)

In [ ]:
Implies(Qx, Px)

In [ ]:
true_for_each_then_true_for_all_conditioned.instantiate(
    {n: n_sub, ExprTuple(var_range_update): var_range_sub, x: x_sub, Px: Px_sub, Qx: Qx_sub},
    num_forall_eliminations=3, assumptions=[Implies(Less(one, zero), Less(one, one))])